In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchsummary import summary
import torchvision

# 1. Setup

In [2]:
# Random seed
np.random.seed(1024)
torch.manual_seed(1024)

In [3]:
# GPU setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)
parallel = True
num_workers = 0

In [4]:
# data setting
columns = [f"col_{i+1}" for i in range(501)] # 65*501
window_size=None
slide_size=None

enlarge_size = (65,65)

# Data main folder
dirc = "E:/external_data/Experiment4/Spectrogram_data_csv_files/CSI_data"

# Saving path
PATH = 'C://Users/Creator/Script/Python/Project/irs_toolbox' # '.'

# Training setting
bsz = 128
pre_train_epochs = 500
fine_tune_epochs = 300

exp_name = 'Encoder_64-128-512-64-7_pretrained_on_exp4csi'

# 2. Pretraining

* `import data` import the csv files and label each file based on its sub-folder. To increase the number sample, each file undergoes data augmentation with `window_size` and `slide_size` . For example, a file with directory: `dirc/label_name/file` and size: (65,501), its form multiple data with label `label_name`.

* We take 20% of the entire data as validation and the remained as training set 


* Previous studies shows the pretraining is benefited from a larger batch size. However, due to GPU Memory limitation, here we are using batch size of 128 


In [5]:
from data.spectrogram import import_data
from data.process_data import label_encode, create_dataloaders

def prepare_dataloader():
    X,y  = import_data(dirc,columns=columns,window_size=window_size,slide_size=slide_size)
    X = X.reshape(*X.shape,1).transpose(0,3,1,2)
    y,lb = label_encode(y)
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
    train_loader, test_loader = create_dataloaders(X_train, y_train, X_test, y_test, train_batch_sizes=bsz, test_batch_sizes=200, num_workers=num_workers)
    return train_loader, test_loader,lb


train_loader, test_loader, lb = prepare_dataloader()

Import Data


For data augmentation, we only use crop, the data is enlarged back t `enlarge_size`

In [6]:
from models.self_supervised import DataAugmentation

def create_pipe():
    # External libraries required
    
    pipe = DataAugmentation(enlarge_size)
    return pipe


data_aug = create_pipe()

Here we use 3 layer CNN as the encoder

In [7]:
from models import Lambda

class Encoder(nn.Module):
    """
    Encoder for spectrogram (1,65,65), 3 layer
    """
    def __init__(self,num_filters):
        super(Encoder, self).__init__()
        l1,l2,l3 = num_filters
        ### 1st ###
        self.conv1 = nn.Conv2d(1,l1,kernel_size=5,stride=2)
        self.norm1 = nn.BatchNorm2d(l1)
        self.actv1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2))
        ### 2nd ###
        self.conv2 = nn.Conv2d(l1,l2,kernel_size=3,stride=2)
        self.norm2 = nn.BatchNorm2d(l2)
        self.actv2 = nn.ReLU()
        self.pool2 = Lambda(lambda x:x) 
        ### 3rd ###
        self.conv3 = nn.Conv2d(l2,l3,kernel_size=2,stride=2)
        self.norm3 = Lambda(lambda x:x)
        self.actv3 = nn.Tanh()
        self.pool3 = nn.AvgPool2d(kernel_size=(2,2))

    def forward(self,X):
        X = self.pool1(self.actv1(self.norm1(self.conv1(X))))
        X = self.pool2(self.actv2(self.norm2(self.conv2(X))))
        X = self.pool3(self.actv3(self.norm3(self.conv3(X))))
        X = torch.flatten(X, 1)
        # print(X.shape)
        return X

A SimCLR implementation, the entire pretrain model consists of an encoder and a projection head. Where the projection head is multilayer perceptron (512>512>128)

In [8]:
from models import ED_module
from models.self_supervised import Projection_head

def create_pretrain_model():
    # External libraries required
    enc = Encoder([64,128,512])
    clf = Projection_head(512,128,head='mlp')
    model = ED_module(encoder=enc,decoder=clf)
    return model

In [17]:
pretrain_model = create_pretrain_model()
summary(pretrain_model,(1,*enlarge_size),batch_size=2*bsz,device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [256, 64, 31, 31]           1,664
       BatchNorm2d-2          [256, 64, 31, 31]             128
              ReLU-3          [256, 64, 31, 31]               0
         MaxPool2d-4          [256, 64, 15, 15]               0
            Conv2d-5           [256, 128, 7, 7]          73,856
       BatchNorm2d-6           [256, 128, 7, 7]             256
              ReLU-7           [256, 128, 7, 7]               0
            Lambda-8           [256, 128, 7, 7]               0
            Conv2d-9           [256, 512, 3, 3]         262,656
           Lambda-10           [256, 512, 3, 3]               0
             Tanh-11           [256, 512, 3, 3]               0
        AvgPool2d-12           [256, 512, 1, 1]               0
          Encoder-13                 [256, 512]               0
           Linear-14                 [2

Here we directly implement **supervised contrastive loss** from [Supervised Contrastive Learning](https://github.com/HobbitLong/SupContrast/tree/6d5a3de39070249a19c62a345eea4acb5f26c0bc), a modification of **NT-Xent(normalized temperature-scaled cross entropy loss)** from SimCLR, where we leveraging the label information for class seperation  

In [18]:
from losses import SupConLoss

def create_criterion():
    # External libraries required
    criterion = SupConLoss(temperature=0.1,base_temperature=0.1)
    return criterion

criterion = create_criterion()

For pretraining, we use SGD with learning rate of $5\cdot 10^{-5}$. For finetuning, we use ADAM with learning rate $1\cdot10^{-5}$ 

In [19]:
def create_optimizer(mode,model):
    if mode == 'pretrain':
        optimizer = torch.optim.SGD(list(model.parameters()), lr=0.0005)
    elif mode == 'finetuning':
        optimizer = torch.optim.Adam(list(model.parameters()), lr=0.0001)
    else:
        raise ValueError("mode: {'pretrain','finetuning'}")
    return optimizer

In [20]:
optimizer = create_optimizer('pretrain',pretrain_model)

Pretraining process

In [21]:
def pretrain(model,train_loader,data_aug,optimizer,criterion,end,start=1,parallel=True):

    # Check device setting
    if parallel == True:
        print('GPU')
        model = model.to(device)
        data_aug = data_aug.to(device)

    else:
        print('CPU')
        model = model.cpu()

    print('Start Training')
    record = {'train':[],'validation':[]}
    i = start

    #Loop
    while i <= end:
        print(f"Epoch {i}: ", end='')
        for b, (X, y) in enumerate(train_loader):
            
            if parallel == True:
                X = X.to(device)
                y = y.to(device)
                
            print(f">", end='')
            optimizer.zero_grad()
            batch_size = X.shape[0]
            # we concatenate the two verison of X at dim0
            X = torch.cat(data_aug(X), dim=0)
            logits = model(X)
            # we split logit with batch size so we get back the two encoded  data
            logits = torch.split(logits, [batch_size, batch_size], dim=0)
            # We concatenate the two version at dim1 
            logits = torch.cat((logits[0].unsqueeze(1),logits[1].unsqueeze(1)),dim=1)
            loss = criterion(logits,y)
            loss.backward()
            optimizer.step()

        # One epoch completed
        l = loss.tolist()
        record['train'].append(l)
        print(f' loss: {l} ')
        i += 1
        del X,y,logits

    model = model.cpu()
    return model,record

pretrain_model, record = pretrain(pretrain_model,train_loader,data_aug,optimizer,criterion,pre_train_epochs,start=1,parallel=parallel)

GPU
Start Training
Epoch 1: >>>>> loss: 5.679641246795654 
Epoch 2: >>>>> loss: 5.8811140060424805 
Epoch 3: >>>>> loss: 5.531131744384766 
Epoch 4: >>>>> loss: 5.546865940093994 
Epoch 5: >>>>> loss: 5.568600654602051 
Epoch 6: >>>>> loss: 5.613371849060059 
Epoch 7: >>>>> loss: 5.523183822631836 
Epoch 8: >>>>> loss: 5.6177802085876465 
Epoch 9: >>>>> loss: 5.570920467376709 
Epoch 10: >>>>> loss: 5.486598014831543 
Epoch 11: >>>>> loss: 5.478726387023926 
Epoch 12: >>>>> loss: 5.480783462524414 
Epoch 13: >>>>> loss: 5.5811076164245605 
Epoch 14: >>>>> loss: 5.489938735961914 
Epoch 15: >>>>> loss: 5.525685787200928 
Epoch 16: >>>>> loss: 5.443608283996582 
Epoch 17: >>>>> loss: 5.496007442474365 
Epoch 18: >>>>> loss: 5.513676643371582 
Epoch 19: >>>>> loss: 5.545963287353516 
Epoch 20: >>>>> loss: 5.520123481750488 
Epoch 21: >>>>> loss: 5.461921691894531 
Epoch 22: >>>>> loss: 5.3948774337768555 
Epoch 23: >>>>> loss: 5.551749229431152 
Epoch 24: >>>>> loss: 5.482010364532471 
Ep

Epoch 198: >>>>> loss: 5.436718940734863 
Epoch 199: >>>>> loss: 5.468601226806641 
Epoch 200: >>>>> loss: 5.348794460296631 
Epoch 201: >>>>> loss: 5.438653945922852 
Epoch 202: >>>>> loss: 5.2752156257629395 
Epoch 203: >>>>> loss: 5.3263959884643555 
Epoch 204: >>>>> loss: 5.309413909912109 
Epoch 205: >>>>> loss: 5.379946231842041 
Epoch 206: >>>>> loss: 5.302030086517334 
Epoch 207: >>>>> loss: 5.317744731903076 
Epoch 208: >>>>> loss: 5.540444374084473 
Epoch 209: >>>>> loss: 5.302358627319336 
Epoch 210: >>>>> loss: 5.38538122177124 
Epoch 211: >>>>> loss: 5.415989875793457 
Epoch 212: >>>>> loss: 5.389938831329346 
Epoch 213: >>>>> loss: 5.369144916534424 
Epoch 214: >>>>> loss: 5.40007209777832 
Epoch 215: >>>>> loss: 5.3577117919921875 
Epoch 216: >>>>> loss: 5.3353495597839355 
Epoch 217: >>>>> loss: 5.280599117279053 
Epoch 218: >>>>> loss: 5.322994709014893 
Epoch 219: >>>>> loss: 5.307888031005859 
Epoch 220: >>>>> loss: 5.273429870605469 
Epoch 221: >>>>> loss: 5.5231871

Epoch 393: >>>>> loss: 5.375254154205322 
Epoch 394: >>>>> loss: 5.28487491607666 
Epoch 395: >>>>> loss: 5.433322906494141 
Epoch 396: >>>>> loss: 5.257129192352295 
Epoch 397: >>>>> loss: 5.119325637817383 
Epoch 398: >>>>> loss: 5.324708938598633 
Epoch 399: >>>>> loss: 5.350872039794922 
Epoch 400: >>>>> loss: 5.150448799133301 
Epoch 401: >>>>> loss: 5.226493835449219 
Epoch 402: >>>>> loss: 5.262145042419434 
Epoch 403: >>>>> loss: 5.314591884613037 
Epoch 404: >>>>> loss: 5.325690269470215 
Epoch 405: >>>>> loss: 5.153660774230957 
Epoch 406: >>>>> loss: 5.123804092407227 
Epoch 407: >>>>> loss: 5.399303436279297 
Epoch 408: >>>>> loss: 5.118254661560059 
Epoch 409: >>>>> loss: 5.20853328704834 
Epoch 410: >>>>> loss: 5.354971885681152 
Epoch 411: >>>>> loss: 5.318849563598633 
Epoch 412: >>>>> loss: 5.17145299911499 
Epoch 413: >>>>> loss: 5.284162521362305 
Epoch 414: >>>>> loss: 5.380411624908447 
Epoch 415: >>>>> loss: 5.462703704833984 
Epoch 416: >>>>> loss: 5.126431465148

Record logging and Saving

In [22]:
from train import make_directory

def record_log(mode,epochs,record,cmtx=None,cls=None):
    if mode == 'pretrain':
        path = make_directory(exp_name+'_pretrain',epoch=epochs,filepath=PATH+'/record/')
        pd.DataFrame(record['train'],columns=['train_loss']).to_csv(path+'_loss.csv')
    elif mode == 'finetuning':
        path = make_directory(exp_name+'_finetuning',epoch=epochs,filepath=PATH+'/record/')
        pd.DataFrame(record['train'],columns=['train_loss']).to_csv(path+'_loss.csv')
        pd.DataFrame(record['validation'],columns=['validation_accuracy']).to_csv(path+'_accuracy.csv')
        cls.to_csv(path+'_report.csv')
        cmtx.to_csv(path+'_cmtx.csv')
    return

In [23]:
record_log('pretrain',pre_train_epochs,record)

In [24]:
from train import save_checkpoint

def save(mode,model,optimizer,epochs):
    if mode == 'pretrain':
        path = make_directory(exp_name+'_pretrain',epoch=epochs,filepath=PATH+'/models/saved_models/')
        save_checkpoint(model, optimizer, epochs, path)
    elif mode == 'finetuning':
        path = make_directory(exp_name+'_finetuning',epoch=epochs,filepath=PATH+'/models/saved_models/')
        save_checkpoint(model, optimizer, epochs, path)
    return

In [ ]:
# save('pretrain',pretrain_model,optimizer,pre_train_epochs)

In [25]:
del criterion, optimizer, record

# 3. Fine-tuning

For finetuning, we only take the encoder from the pretrain model, freeze the encoder and build a classifier (512>64>7) upon it.

In [26]:
from models import ED_module, Classifier

def freeze_network(model):
    for _, p in model.named_parameters():
        p.requires_grad = False
    return model

def create_finetune_model(enc=None):
    # External libraries required
    if enc == None:
        enc = Encoder([64,128,512])
    else:
        enc = freeze_network(enc)
    clf = Classifier(512,64,7)
    model = ED_module(encoder=enc,decoder=clf)
    return model


# finetune_model = create_finetune_model(None)
finetune_model = create_finetune_model(pretrain_model.encoder)
criterion = nn.CrossEntropyLoss()
optimizer = create_optimizer('finetuning',finetune_model)

In [15]:
summary(finetune_model,(1,*enlarge_size),batch_size=bsz,device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [128, 64, 31, 31]           1,664
       BatchNorm2d-2          [128, 64, 31, 31]             128
              ReLU-3          [128, 64, 31, 31]               0
         MaxPool2d-4          [128, 64, 15, 15]               0
            Conv2d-5           [128, 128, 7, 7]          73,856
       BatchNorm2d-6           [128, 128, 7, 7]             256
              ReLU-7           [128, 128, 7, 7]               0
            Lambda-8           [128, 128, 7, 7]               0
            Conv2d-9           [128, 512, 3, 3]         262,656
           Lambda-10           [128, 512, 3, 3]               0
             Tanh-11           [128, 512, 3, 3]               0
        AvgPool2d-12           [128, 512, 1, 1]               0
          Encoder-13                 [128, 512]               0
           Linear-14                  [

In [ ]:
from sklearn.metrics import accuracy_score

def short_evaluation(model,data_aug,test_loader,parallel=True):
    # copy the model to cpu
    if parallel == True:
        model = model.cpu()
        data_aug = data_aug.cpu()
    with torch.no_grad():
        for X_test, y_test in test_loader:
            X_test,_ = data_aug(X_test)
            y_val = model(X_test)
            predicted = torch.max(y_val, 1)[1]
            acc = accuracy_score(y_test.view(-1), predicted.view(-1))
    # send model back to gpu
    if parallel == True:
        model = model.cuda()
    return acc


def finetuning(model, data_aug, train_loader, criterion, optimizer, end, start = 1, test_loader = None, parallel = None, **kwargs):

    # Check device setting
    if parallel == True:
        print('GPU')
        model = model.to(device)
        data_aug = data_aug.to(device)
    else:
        print('CPU')

    print('Start Training')
    record = {'train':[],'validation':[]}
    i = start
    #Loop
    while i <= end:
        print(f"Epoch {i}: ", end='')
        for b, (X_train, y_train) in enumerate(train_loader):

            if parallel == True:
                X_train = X_train.cuda() #.to(device)

            X_train,_ = data_aug(X_train)

            print(f">", end='')
            optimizer.zero_grad()
            y_pred = model(X_train)

            if parallel == True:
                X_train = X_train.cpu()
                del X_train
                y_train = y_train.cuda()

            loss   = criterion(y_pred, y_train)
            loss.backward()
            optimizer.step()

            if parallel == True:
                y_pred = y_pred.cpu()
                y_train = y_train.cpu()
                del y_pred,y_train

        # One epoch completed
        loss = loss.tolist()
        record['train'].append(loss)
        print(f' loss: {loss} ',end='')
        if (test_loader != None) and i%10 ==0 :
            acc = short_evaluation(model,data_aug,test_loader,parallel)
            record['validation'].append(acc)
            print(f' accuracy: {acc}')
        else:
            print('')
        i += 1

    model = model.cpu()
    return model, record

finetune_model, record = finetuning(finetune_model , data_aug, train_loader, criterion, optimizer, fine_tune_epochs, 1, test_loader, parallel)

In [ ]:
from train import evaluation

cmtx,cls = evaluation(finetune_model,test_loader,label_encoder=lb)

In [ ]:
record_log('finetuning',fine_tune_epochs,record,cmtx,cls)

In [ ]:
save('finetuning',finetune_model,optimizer,fine_tune_epochs)

# Record

In [20]:
# Normal
finetune_model, record = finetuning(finetune_model , data_aug, train_loader, criterion, optimizer, fine_tune_epochs, 1, test_loader, parallel)

GPU
Start Training
Epoch 1: >>>>> loss: 1.1421394348144531 
Epoch 2: >>>>> loss: 1.09224271774292 
Epoch 3: >>>>> loss: 1.2326371669769287 
Epoch 4: >>>>> loss: 1.1167579889297485 
Epoch 5: >>>>> loss: 1.3625119924545288 
Epoch 6: >>>>> loss: 1.1908559799194336 
Epoch 7: >>>>> loss: 1.076444387435913 
Epoch 8: >>>>> loss: 1.328795075416565 
Epoch 9: >>>>> loss: 1.1221295595169067 
Epoch 10: >>>>> loss: 1.0692933797836304  accuracy: 0.531578947368421
Epoch 11: >>>>> loss: 1.3332382440567017 
Epoch 12: >>>>> loss: 1.4822489023208618 
Epoch 13: >>>>> loss: 1.3546075820922852 
Epoch 14: >>>>> loss: 1.2067177295684814 
Epoch 15: >>>>> loss: 1.124380111694336 
Epoch 16: >>>>> loss: 0.9782644510269165 
Epoch 17: >>>>> loss: 1.0654231309890747 
Epoch 18: >>>>> loss: 1.2197911739349365 
Epoch 19: >>>>> loss: 1.1384150981903076 
Epoch 20: >>>>> loss: 1.2830332517623901  accuracy: 0.48947368421052634
Epoch 21: >>>>> loss: 1.2567675113677979 
Epoch 22: >>>>> loss: 1.2567341327667236 
Epoch 23: >>>

Epoch 183: >>>>> loss: 1.2118840217590332 
Epoch 184: >>>>> loss: 1.3489304780960083 
Epoch 185: >>>>> loss: 1.257980465888977 
Epoch 186: >>>>> loss: 0.28101927042007446 
Epoch 187: >>>>> loss: 0.3816763758659363 
Epoch 188: >>>>> loss: 0.3407120108604431 
Epoch 189: >>>>> loss: 1.3119910955429077 
Epoch 190: >>>>> loss: 1.078587293624878  accuracy: 0.7368421052631579
Epoch 191: >>>>> loss: 0.3806036412715912 
Epoch 192: >>>>> loss: 1.2969286441802979 
Epoch 193: >>>>> loss: 1.1460391283035278 
Epoch 194: >>>>> loss: 1.5167796611785889 
Epoch 195: >>>>> loss: 1.240458607673645 
Epoch 196: >>>>> loss: 1.3349157571792603 
Epoch 197: >>>>> loss: 0.4032554030418396 
Epoch 198: >>>>> loss: 1.1011329889297485 
Epoch 199: >>>>> loss: 0.3816370666027069 
Epoch 200: >>>>> loss: 0.320182204246521  accuracy: 0.6842105263157895
Epoch 201: >>>>> loss: 0.3059037923812866 
Epoch 202: >>>>> loss: 0.3218662440776825 
Epoch 203: >>>>> loss: 0.28663450479507446 
Epoch 204: >>>>> loss: 0.2610678076744079

In [27]:
# Pretrained
finetune_model, record = finetuning(finetune_model , data_aug, train_loader, criterion, optimizer, fine_tune_epochs, 1, test_loader, parallel)

GPU
Start Training
Epoch 1: >>>>> loss: 1.8875242471694946 
Epoch 2: >>>>> loss: 1.855168342590332 
Epoch 3: >>>>> loss: 1.8436195850372314 
Epoch 4: >>>>> loss: 1.826266884803772 
Epoch 5: >>>>> loss: 1.8050285577774048 
Epoch 6: >>>>> loss: 1.7451403141021729 
Epoch 7: >>>>> loss: 1.6992255449295044 
Epoch 8: >>>>> loss: 1.72196364402771 
Epoch 9: >>>>> loss: 1.6971665620803833 
Epoch 10: >>>>> loss: 1.7223783731460571  accuracy: 0.43157894736842106
Epoch 11: >>>>> loss: 1.6832969188690186 
Epoch 12: >>>>> loss: 1.6404898166656494 
Epoch 13: >>>>> loss: 1.6534546613693237 
Epoch 14: >>>>> loss: 1.6362974643707275 
Epoch 15: >>>>> loss: 1.5436838865280151 
Epoch 16: >>>>> loss: 1.4615058898925781 
Epoch 17: >>>>> loss: 1.6109257936477661 
Epoch 18: >>>>> loss: 1.5991570949554443 
Epoch 19: >>>>> loss: 1.5764433145523071 
Epoch 20: >>>>> loss: 1.562514305114746  accuracy: 0.45789473684210524
Epoch 21: >>>>> loss: 1.4584593772888184 
Epoch 22: >>>>> loss: 1.3952381610870361 
Epoch 23: >

Epoch 183: >>>>> loss: 1.0556892156600952 
Epoch 184: >>>>> loss: 1.167283296585083 
Epoch 185: >>>>> loss: 1.344879150390625 
Epoch 186: >>>>> loss: 1.128980040550232 
Epoch 187: >>>>> loss: 1.4071509838104248 
Epoch 188: >>>>> loss: 1.3462942838668823 
Epoch 189: >>>>> loss: 1.051360845565796 
Epoch 190: >>>>> loss: 1.259808897972107  accuracy: 0.5421052631578948
Epoch 191: >>>>> loss: 1.0031287670135498 
Epoch 192: >>>>> loss: 1.2465404272079468 
Epoch 193: >>>>> loss: 1.0576963424682617 
Epoch 194: >>>>> loss: 1.3436250686645508 
Epoch 195: >>>>> loss: 1.2193162441253662 
Epoch 196: >>>>> loss: 1.1716508865356445 
Epoch 197: >>>>> loss: 1.0499998331069946 
Epoch 198: >>>>> loss: 1.1551618576049805 
Epoch 199: >>>>> loss: 0.9820274114608765 
Epoch 200: >>>>> loss: 1.4822421073913574  accuracy: 0.5105263157894737
Epoch 201: >>>>> loss: 1.2301011085510254 
Epoch 202: >>>>> loss: 1.2018553018569946 
Epoch 203: >>>>> loss: 1.0777796506881714 
Epoch 204: >>>>> loss: 1.051023006439209 
Ep